## Super Bowl Squares Average Payout Analysis

Gian Favero | February 2nd, 2024

We look to Stathead.com's DriveFinder to estimate the probability of any scoring event (TD, FG, safety) occurring during a given drive. We assume each drive is independent of another (no accommodations for game script) and will handle the scenario of PATs as a double scoring drive should a TD be the result of a drive. Historical PAT data shows nearly 100% success on average over the past ten years, so the chance of incurring a PAT is equal to the chance of scoring a TD. 

We additionally look to Pro Football Reference to determine how many drives on average each team gets in a given NFL game. We multiply this number by two to get the total number of offensive drives per game

In [175]:
import numpy as np

buy_in = 2
payout = 0.05
pot = 100 * buy_in

# Stats taken from Stathead.com Drive Finder over 2013 - 2023 seasons
p_TD = 0.213
p_FG = 0.144
p_S = 0.002
no_score = 1 - p_TD - p_FG - p_S

drive_probabilities = [p_TD, p_FG, p_S, no_score]

# Stat taken from Pro Football Reference over 2013 - 2023 seasons
drives_per_game = 11.5 * 2

Using this information, we now have enough data to run Monte Carlo simulations to determine the number of times each square will be landed on in a given NFL game. We consider each drive independently (again, not accounting for game script) and consider that a PAT occurs after each TD at a 100% conversion rate.

In [176]:
simulations = 10000
square_board = np.zeros((10, 10))

for i in range(simulations):

    team_1_score = 0
    team_2_score = 0

    for j in range(int(drives_per_game)):
        # Reset score for each drive
        score = 0

        # Select drive outcome based on probabilities
        drive_outcome = np.random.choice([0, 1, 2, 3], p=drive_probabilities)

        # Add score based on drive outcome
        if drive_outcome == 0:      # TD
            score += 6
        elif drive_outcome == 1:    # FG
            score += 3
        elif drive_outcome == 2:    # Safety
            score += 2
        else:                       # No score
            score = 0

        # Add score to team 1 or team 2 based on drive number
        if j % 2 == 0:
            team_1_score += score
        else:
            team_2_score += score

        # Update board if score occurred
        if score > 0:
            square_board[team_1_score % 10][team_2_score % 10] += 1

            # If result was a touchdown, add the extra point
            if drive_outcome == 0:
                if j % 2 == 0:
                    team_1_score += 1
                else:
                    team_2_score += 1

                square_board[team_1_score % 10][team_2_score % 10] += 1

# Normalize the board
square_board = square_board / simulations

# Print the board
print("Times landed per square: ")
np.set_printoptions(suppress=True)
print(square_board)

# Print the sum of the board
print("\nSum of the number of landings: ")
print(np.sum(square_board))

Times landed per square: 
[[0.564 0.128 0.049 0.589 0.301 0.03  0.489 0.658 0.048 0.148]
 [0.142 0.048 0.015 0.113 0.085 0.011 0.07  0.133 0.019 0.037]
 [0.057 0.017 0.004 0.039 0.033 0.003 0.021 0.051 0.006 0.01 ]
 [0.688 0.106 0.036 0.4   0.241 0.022 0.259 0.472 0.038 0.095]
 [0.343 0.093 0.031 0.256 0.181 0.02  0.162 0.288 0.036 0.075]
 [0.032 0.011 0.004 0.024 0.023 0.003 0.017 0.031 0.005 0.007]
 [0.629 0.067 0.02  0.233 0.151 0.013 0.11  0.307 0.025 0.046]
 [0.788 0.127 0.047 0.467 0.276 0.026 0.346 0.534 0.051 0.123]
 [0.056 0.02  0.007 0.044 0.037 0.006 0.026 0.053 0.01  0.013]
 [0.166 0.033 0.009 0.104 0.078 0.008 0.049 0.139 0.013 0.019]]

Sum of the number of landings: 
13.190300000000002


This probability per square graphic is essentially telling us how many times an individual square is expected to be landed on per game. We can then get the expected earnings per square through the expected payout per score.

In [177]:
# Calculate the payout per square
payout_board = square_board * payout * pot
print("Payout per square: ")
print(payout_board)

Payout per square: 
[[5.641 1.28  0.486 5.887 3.012 0.299 4.89  6.578 0.483 1.479]
 [1.419 0.479 0.149 1.125 0.852 0.108 0.696 1.328 0.194 0.366]
 [0.574 0.166 0.042 0.387 0.325 0.029 0.211 0.512 0.063 0.099]
 [6.877 1.059 0.362 4.001 2.407 0.221 2.594 4.723 0.38  0.945]
 [3.434 0.935 0.308 2.561 1.815 0.199 1.622 2.88  0.361 0.751]
 [0.32  0.11  0.035 0.244 0.227 0.029 0.166 0.307 0.052 0.066]
 [6.287 0.666 0.199 2.333 1.514 0.127 1.104 3.07  0.246 0.456]
 [7.883 1.269 0.471 4.665 2.764 0.265 3.465 5.337 0.506 1.226]
 [0.563 0.204 0.066 0.445 0.375 0.063 0.263 0.527 0.096 0.129]
 [1.658 0.328 0.086 1.037 0.781 0.076 0.486 1.391 0.135 0.191]]


We can additionally estimate the expected grand payout per square. Considering the full pot and an average of 13 payouts per game, the average grand prize will be 

$\text{GP} = \text{Pot} - \text{Payout} \times 13$.

 We have the average number of landings per square in a given game, so if we divide this by 13 we will have the probability of each square being landed on for one individual score in that game (which could be the final score). We multiply this by the expected grand prize to approximate the expected grand prize per square.

In [178]:
# Add a board that represents winning the grand prize according to an average of 13 payouts per game
normalized_board = square_board / 13
grand_board = normalized_board * (pot - pot * payout * 13)

# Print the grand prize board with 3 decimal places
print("Grand prize board: ")
np.set_printoptions(precision=3)
print(grand_board)

Grand prize board: 
[[3.037 0.689 0.262 3.17  1.622 0.161 2.633 3.542 0.26  0.796]
 [0.764 0.258 0.08  0.606 0.459 0.058 0.375 0.715 0.104 0.197]
 [0.309 0.089 0.023 0.208 0.175 0.016 0.114 0.276 0.034 0.053]
 [3.703 0.57  0.195 2.154 1.296 0.119 1.397 2.543 0.205 0.509]
 [1.849 0.503 0.166 1.379 0.977 0.107 0.873 1.551 0.194 0.404]
 [0.172 0.059 0.019 0.131 0.122 0.016 0.089 0.165 0.028 0.036]
 [3.385 0.359 0.107 1.256 0.815 0.068 0.594 1.653 0.132 0.246]
 [4.245 0.683 0.254 2.512 1.488 0.143 1.866 2.874 0.272 0.66 ]
 [0.303 0.11  0.036 0.24  0.202 0.034 0.142 0.284 0.052 0.069]
 [0.893 0.177 0.046 0.558 0.421 0.041 0.262 0.749 0.073 0.103]]


We can now introduce the league constraints to the problem. Each square costs $2 for a maximum of ten squares. Squares are assigned randomly according to a uniform distribution. Using this information, we can attempt to discern if buying a particular number of squares maximizes our chances of breaking even.

In [181]:
simulations = 50000

# Run a loop over each purchasable number of squares
for i in range(10):
    payout = 0
    # Run a Monte Carlo simulation over a number of trials
    for j in range(simulations):
        # Randomly select i + 1 squares from the board
        team_1_index = np.random.choice(10, i+1)
        team_2_index = np.random.choice(10, i+1)

        # Sum the expected payouts for each square
        for k in range(i+1):
            payout += payout_board[team_1_index[k]][team_2_index[k]] + grand_board[team_1_index[k]][team_2_index[k]]

    # Print the expected payout for each number of squares
    print(f"Expected payout for {i+1} squares: {payout/simulations}, Profit: {payout/simulations - 2 * (i+1)}")

Expected payout for 1 squares: 2.0088970461538826, Profit: 0.008897046153882648
Expected payout for 2 squares: 4.053459538461815, Profit: 0.05345953846181484
Expected payout for 3 squares: 6.09163233846161, Profit: 0.09163233846160956
Expected payout for 4 squares: 8.103293846153349, Profit: 0.10329384615334902
Expected payout for 5 squares: 10.19164381538334, Profit: 0.1916438153833404
Expected payout for 6 squares: 12.106351815382565, Profit: 0.1063518153825651
Expected payout for 7 squares: 14.162494276920436, Profit: 0.1624942769204356
Expected payout for 8 squares: 16.21169341538096, Profit: 0.2116934153809602
Expected payout for 9 squares: 18.25570153845728, Profit: 0.2557015384572807
Expected payout for 10 squares: 20.26739972307131, Profit: 0.26739972307131055


We observe a clear bias towards those who buy as many squares as possible. Thus, if you decide to buy into a Super Bowl pool this year, go all in!